## Лабораторная работа №3. Методы классификации данных.

#### Выполнила:
Плохута Д.Г.  
#### Группа: 
19-ИВТ-3  
#### Вариант:
№6  
#### Набор данных:
6 (occupancy)
#### Классификаторы:  
2 - классификатор K ближайших соседей (задаётся радиус для выбора ближайших объектов);  
4 - логический классификатор (бинарное решающее дерево).  

### 1.	Изучить набор данных. Создать описание набора данных на русском языке. Описать признаки, используемые в наборе данных (включить полученные описания в отчёт).

1. date - дата и время замера
2. Temperature - температура в градусах Цельсия 
3. Humidity - влажность воздуха, %
4. Light - освещенность офиса, Люкс
5. CO2 - уровень углекислого газа CO2, ppm (пропромилле)
6. Humidity Ratio - удельная влажность, производное количество от температуры и относительной влажности, в кг водяного пара/кг воздуха
7. Occupancy - заполненность, 0 или 1: 0 для незанятого, 1 для занятого состояния

### 2.	Удалите дубликаты строк в наборе данных; приведите размер набора данных до и после данной операции.


In [4]:
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import numpy as np

In [5]:
#occupancy_DF = pd.read_csv('C:\\Users\\Acer\\Desktop\\уник\\мо\\лб2_классификация\\6_occupancy\\read.txt')
#occupancy_DF

In [6]:
init_len= len(#occupancy_DF)
#occupancy_DF.drop_duplicates(keep= False,inplace=True)
finit_len = len(#occupancy_DF)
print("Удалено дубликатов: " , init_len - finit_len)

SyntaxError: unexpected EOF while parsing (<ipython-input-6-df411adbd197>, line 4)

In [ ]:
#occupancy_DF

### 3.	Оцените сбалансированность данных по классам (постройте гистограмму). Используйте полученную информацию при выборе метрики оценки качества классификации (PR или ROC кривая).

In [ ]:
import seaborn as sns

In [ ]:
occupancy_DF['Occupancy'].value_counts().plot(kind='bar', label='Occupancy')
plt.legend()
plt.title("Занятось офиса");

Как видно из гистограммы, классы несбалансированны, преобладает первый класс (0).

### 4.	Выполните масштабирование количественных признаков; Постройте диаграммы BoxPlot для признаков до и после масштабирования. Выберите способ масштабирования (например, нормализацию или стандартизацию).

In [ ]:
before_scaling = ["Temperature", "Humidity", "Light", "CO2", "HumidityRatio"]
sns.boxplot(data = occupancy_DF[before_scaling])

#### Нормализация

In [ ]:
import sklearn as skl
from sklearn.preprocessing import MinMaxScaler

In [ ]:
occupancy_DF[before_scaling] = MinMaxScaler().fit_transform(occupancy_DF[before_scaling])
sns.boxplot(data = occupancy_DF[before_scaling])

### 5.	Выполните замену категориальных признаков; выберите и обоснуйте способ замены.

In [ ]:
categorical = [c for c in occupancy_DF.columns if occupancy_DF[c].dtype.name == 'object']
print("Категориальные признаки: ", categorical)

In [ ]:
occupancy_DF[categorical].describe()

In [ ]:
occupancy_DF['date'].unique()

#### Все значения уникальны (указано в сопровождающем документе, измерения проводились каждую минуту), замена не имеет смысла

### 6.	Оцените корреляцию между признаков и удалите те признаки, которые коррелируют с наибольшим числом других (удалять признаки нужно только для линейных методов классификации).

In [ ]:
corr_matrix = occupancy_DF.corr()
sns.heatmap(corr_matrix)

#### Признаки коррелируют равномерно

### 7.	Заполните пропущенные значения в данных.

In [ ]:
occupancy_DF.count()

#### Пропущенных значений нет

### 8.	Решите поставленную задачу классификации в соответствии с заданием. 
#### При подборе параметров классификатора используйте метод GridSearchCV и перекрёстную проверку (изучите возможные для изменения параметры классификации). Определите схему построения многоклассового классификатора, используемую по умолчанию (опишите используемую схему кодирования, обоснуйте свой выбор). Постройте, если это возможно, многоклассовую классификацию на основе схем «один-против-всех» и «все-против-всех». Оцените точность классификации для каждой их схем. Постройте кривые PR и ROC (для каждого из классов должны быть построены отдельные кривые, а также кривые для микро и макроусреднения метрик качества). Для линейного классификатора используйте регуляризацию. 

#### 1) Классификатор K ближайших соседей (задаётся радиус для выбора ближайших объектов):

In [ ]:
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
X = occupancy_DF.drop(('date'), axis=1)
Y = occupancy_DF.Occupancy
X.columns 

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)

In [ ]:
radius_classif = RadiusNeighborsClassifier()
radius_classif.get_params()
params = {'radius': [2, 3, 4],
          'weights': ['uniform', 'distance']}
radius_classif_grid = GridSearchCV(radius_classif, params)
radius_classif_grid.fit(x_train, y_train)

In [ ]:
#r = radius_classif_grid.best_estimator_.radius_neighbors
radius_classif = RadiusNeighborsClassifier(4)
radius_classif.fit(x_train, y_train)

rad_classif_predict = radius_classif.predict(x_test)

print('Accuracy: ', accuracy_score(y_test, rad_classif_predict ))
print('Best: ',  radius_classif_grid.best_score_)

#### 2) Логический классификатор (бинарное решающее дерево):

In [ ]:
from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier()

parametrs = {'max_depth': list(range(1, 7)), 'max_features': list(range(1, 7))}
decision_tree_grid = GridSearchCV(decision_tree, parametrs)
decision_tree_grid.fit(x_train, y_train)

In [ ]:
best_features = decision_tree_grid.best_estimator_.max_features
best_depth = decision_tree_grid.best_estimator_.max_depth
best_score = decision_tree_grid.best_score_
print('best train score:', best_score, 
      'features number:', best_features,
      'depth:', best_depth)

decision_tree = DecisionTreeClassifier(max_depth=best_depth, max_features=best_features)
decision_tree.fit(x_train, y_train)
best_score  = np.mean(y_test == decision_tree.predict(x_test))
print('test score: ', best_score)

### 9.	Сравните кривые для классификаторов, указанных в задании, сделайте выводы.

In [ ]:
proba_radius_classif = radius_classif.predict_proba(x_test)
proba_decision_tree = decision_tree.predict_proba(x_test)

inversion = [1-y for y in y_test]

##### PR-графики

In [ ]:
def PRPlot(prob, occupancy, label, test):
    precision, recall, _ = precision_recall_curve(test, prob[:, occupancy])
    aps = average_precision_score(test, prob[:, occupancy])
    plt.plot(recall, precision, label='%s PR (area = %f)' % (label, aps))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, average_precision_score
plt.figure(figsize=(10,8))

PRPlot(proba_radius_classif, 0, 'KRadiusNeighbors',inversion)
PRPlot(proba_decision_tree, 0, 'DecisionTree', inversion)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.title('Occupancy = 0')
plt.legend(loc=4, fontsize='medium')
plt.show()

In [ ]:
plt.figure(figsize=(10,8))

PRPlot(proba_radius_classif, 1, 'KRadiusNeighbors', y_test)
PRPlot(proba_decision_tree, 1, 'DecisionTree', y_test)

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.title('Occupancy = 1')
plt.legend(loc=4, fontsize='medium')
plt.show()

##### ROC - графики

In [ ]:
def ROCPlot0(prob, label, test):
    tpr, fpr, _ = roc_curve(test, prob[:, 0])
    roc_auc  = auc(fpr, tpr)
    plt.plot(fpr, tpr, label='%s ROC (area = %f)' % (label, roc_auc)) 

def ROCPlot1(prob, label, test):
    fpr, tpr, _ = roc_curve(test, prob[:, 1])
    roc_auc  = auc(fpr, tpr)
    plt.plot(fpr, tpr, label='%s ROC (area = %f)' % (label, roc_auc))

In [7]:
from sklearn.metrics import roc_curve, auc
plt.figure(figsize=(10,8))

ROCPlot0(proba_radius_classif, 'KRadiusNeighbors', y_test)
ROCPlot0(proba_decision_tree, 'DecisionTree', y_test)

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc=4, fontsize='medium')
plt.title('Occupancy = 0')
plt.show()

NameError: name 'ROCPlot0' is not defined

<Figure size 720x576 with 0 Axes>

In [ ]:
plt.figure(figsize=(10,8))

ROCPlot1(proba_radius_classif, 'KRadiusNeighbors', y_test)
ROCPlot1(proba_decision_tree, 'DecisionTree', y_test)

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([-0.05, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc=4, fontsize='medium')
plt.title('Occupancy = 1')
plt.show()